In [1]:
import nengo
import nengo_spa as spa
import pytry
import numpy as np
from random import shuffle
import random
import sys, os
import math
from IPython import display
from scipy.optimize import brute, minimize

import skopt
from skopt import gp_minimize
from skopt.space.space import Integer, Real
from skopt.plots import plot_evaluations, plot_objective

from scipy.stats import sem
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.style as style
import pickle
import os

from itertools import product



# Import our classes
sys.path.append('..')
import experiments as xps
from experiments import create_xp
from model import ExperimentRun, create_vocab
from data import Data

# Optimization and model parameters

In [2]:
D = int(96)  # the dimensionality of the vectors
PROC_FDBCK = .9
PROC_FDBCK_SYN = .005
GW_FDBCK = 1
GW_SCALE = 20 # scale the input of GW for saturation and catching very short visual stimuli
BG_THR = .1
BG_BIAS = .5
STIM_DURATION = .029
FIX_DURATION = .5
N_NEURONS_SCALE = 1
N_NEURONS_SCALE_COMBINED = .25
INTEGRATOR_RESET = False
N_SAMPLES = 10000

STARTING_SEED = 1
N_SEEDS = 10

N_BLOCKS_PER_OPERATION = 1 # default: 10
N_TRIALS_PER_DIGIT = 2 # default: 5
N_DIFFERENT_DIGITS = 4 # default: 4
N_DIFFERENT_OPERATIONS = 3 # default: 3

data_dir = "evaluate_fit/big_optimization_2"


In [ ]:
def evaluation_function(params):
    
    print("params:", params)
    data = []
    
    D = params[6]*16
    
#     return 0
        
    print('Simulating...')
    for i,seed in enumerate(range(STARTING_SEED, STARTING_SEED+N_SEEDS)):
        print('\t seed '+str(i+1)+'/'+str(N_SEEDS))
        data_filename = 'run_seed'+str(seed)+'_crosstalk'+str(params[0])+'_sevid'+str(params[1])+'_nsamples'+str(params[3])+'_combscale'+str(params[4])+'_procfb'+str(params[5])+'_D'+str(D)+'_gwscale'+str(params[7])+'_prevsyn'+str(params[8])      
        
        try:
            seed_data = pd.read_pickle(data_dir+'/'+data_filename+'.csv')

        except FileNotFoundError:
            xp = create_xp(N_BLOCKS_PER_OPERATION, N_TRIALS_PER_DIGIT, N_DIFFERENT_DIGITS, N_DIFFERENT_OPERATIONS, STIM_DURATION, FIX_DURATION, seed)
            results = ExperimentRun().run(
                n_neurons_scale=N_NEURONS_SCALE,
                n_neurons_scale_combined=params[4],
                s_crosstalk=params[0],
                s_evidence=params[1],
                n_samples=int(params[3]),
                t_senso=0,
                vocab=create_vocab(D, seed),
                xp=xp,
                integrator_reset=INTEGRATOR_RESET,
                proc_feedback=params[5],
                proc_feedback_synapse=PROC_FDBCK_SYN,
                prev_feedback_synapse=params[8],
                GW_feedback=GW_FDBCK,
                GW_scale=params[7],
                BG_thr=BG_THR,
                BG_bias=BG_BIAS,
                seed=seed,
                backend="nengo_ocl",
                data_dir=data_dir,
                data_filename=data_filename,
                plt=False
            )

            seed_data = results['data'].df

            seed_data.to_pickle(data_dir+'/'+data_filename+'.csv')
        data.append(seed_data)
        
        
    data = Data(pd.concat(data))
    data.df['rt'] += params[2]*1000
    error = data.mean_differences_error(compare_errorrates=True, compare_RTs=True, tasks=range(N_DIFFERENT_OPERATIONS))
    print('rmse:', error)
    print('error rate:', data.error_rate)
#     data.plot_fig2_simple(plot_humans=True)
#     data.plot_fig2_chained(plot_humans=True)
    return error

res = gp_minimize(evaluation_function, 
    dimensions=[
        Real(0, 1, name="crosstalk"), # crosstalk
        Real(5, 20, name="evidence strength"), # evidence strength
        Real(0, .2, name="senso"), # sensory delay
        Integer(100, 10000, name="number of samples"), # number of samples
        Real(.1, 1, name="scale combined"), # n_neurons_scale_combined
        Real(.5, 1, name="processor feedback"), # processor feedback
        Integer(2, 10, name="dimensions"), # dimensions (*16)
        Real(1, 30, name="GW scale"), # GW scale
        Real(.002, .02, name="PREV feedback synapse"), # PREV feedback synapse
    ],
    x0=[
        .15, # crosstalk
        15, # evidence strength
        .1, # sensory delay
        N_SAMPLES,
        N_NEURONS_SCALE_COMBINED,
        PROC_FDBCK,
        int(D/16),
        GW_SCALE,
        .01 # prev fb syn
    ],
    n_calls=100,
    random_state=STARTING_SEED
               
)

print(res)

with open(data_dir+"/res.pickle", 'wb') as handle:
    pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

params: [0.15, 15, 0.1, 10000, 0.25, 0.9, 6, 20, 0.01]
Simulating...
	 seed 1/10
running run_seed1_crosstalk0.15_sevid15_nsamples10000_combscale0.25_procfb0.9_D96_gwscale20_prevsyn0.01
